In [1]:
from __future__ import print_function
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils
import pandas as pd
from sklearn import cross_validation
from keras import regularizers
from keras.callbacks import ModelCheckpoint
import os
from collections import Counter
import operator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe

C:\Users\kenneth\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\kenneth\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# network and training
NB_EPOCH = 50
#BATCH_SIZE = 254
#BATCH_SIZE ={{choice([128,512])}}
VERBOSE = 2
#NB_CLASSES = 2   # number of outputs = number of digits
# OPTIMIZER = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000001, amsgrad=False) 
#Adagrad(lr=0.01, epsilon=None, decay=0.0)
#OPTIMIZER=Adadelta(lr=0.8, rho=0.95, epsilon=None, decay=0.0)
#gradient clipping clipnorm=1.  maximum norm of 1.
#clipvalue=0.5 ......max=0.5 and min=-0.5
OPTIMIZER = Adam(lr=3e-4) 
#OPTIMIZER ={{choice(['rmsprop', 'adam','sgd'])}}
N_HIDDEN = 254
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.5
MODEL_DIR = 'model/'

In [3]:
def data():
    x=pd.read_csv('df.csv', index_col=0, parse_dates=True)
    x_pred = pd.read_csv('x_pred.csv', index_col=0, parse_dates=True)
    from sklearn.preprocessing import StandardScaler
    x_scaler=StandardScaler()
    x = x_scaler.fit_transform(x)
    x_pred = x_scaler.fit_transform(x_pred)
    y=pd.read_csv('y.csv', index_col=0, parse_dates=True, names=['label'])
    x_train, x_test, y_train, y_test=cross_validation.train_test_split(x,y,test_size=0.1, random_state=54)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_pred = x_pred.astype('float32')
    NB_CLASSES = 2 
    y_train = np_utils.to_categorical(y_train, NB_CLASSES)
    y_test = np_utils.to_categorical(y_test, NB_CLASSES)
    return x_train, y_train, x_test, y_test
    

In [4]:
def create_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Dense({{choice([64,128, 256, 512])}},input_shape=(x_train.shape[1],),
              kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}),
              bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}))
             )
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([64,128, 256, 512])}}), 
#               kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05])}}),
#               bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05])}})
             )
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([64,128, 256, 512])}}), 
#               kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05])}}),
#               bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05])}})
             )
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    


    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], 
                  optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, 
                                 decay=0.000001, amsgrad=False) 
                 )

    result = model.fit(x_train, y_train,
              batch_size={{choice([128, 254, 512])}},
              epochs=100,
              verbose=0,
              validation_split=0.1)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [5]:
if __name__ == '__main__':
   
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='testing_hyperas_2'
                                         )
    x_train, y_train, x_test, y_test  = data()

    print("Evalutation of best performing model:")
    print(best_model.evaluate(x_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Activation, Dropout
except:
    pass

try:
    from keras.layers import BatchNormalization
except:
    pass

try:
    from keras.optimizers import SGD, RMSprop, Adam, Adadelta
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn import cross_validation
except:
    pass

try:
    from keras import regularizers
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint
except:
    pass

try:
    import os
except:
    pass

try:
    from collections import Counter
except:
    pass

try:
    import operator
except:
    pass

try:
    from sklearn.metrics import roc_auc_score
except:
    pass

try:
    from sklearn.metrics import confusion_matrix
except:
    pass

try: